# Sentiment Analysis of Restaurant Reviews

### This script performs sentiment analysis on a collection of restaurant reviews using machine learning techniques. Sentiment analysis aims to determine the sentiment expressed in text, specifically whether a review conveys a positive or negative sentiment about a restaurant experience.

### Steps:
1. Data Loading and Exploration: The script loads a dataset containing restaurant reviews from a TSV file.It checks for any missing values and provides a glimpse of the dataset's structure.

2. Data Preprocessing: The text data in the reviews undergoes preprocessing to ensure consistency and remove irrelevant information. The text is converted to lowercase, tokenized into words, and stemmed to their root forms. Common stopwords are also removed to focus on meaningful content.

3. Feature Extraction: To enable machine learning models to process text data, the preprocessed reviews are transformed into numerical features. This process, known as Count vectorization, assigns weights to words based on their importance within each review and across the entire dataset.

4. Model Selection and Evaluation: The script considers multiple machine learning models for sentiment analysis, including Multinomial Naive Bayes, Random Forest, Gradient Boosting, XG Boost and Support Vector Classifier (SVC).Each model is trained on a subset of the data and evaluated on another subset to measure its predictive ability.

5. Best Model Identification: During evaluation, the script identifies the best-performing model based on its accuracy in predicting sentiment. This helps determine which model is most effective for this analysis.

6. Evaluation Metrics and Insights: The script provides evaluation metrics such as accuracy, which reflects the proportion of correctly predicted sentiments. Additionally, it generates a confusion matrix and a classification report to offer insights into model performance for positive and negative sentiments.

7. Conclusion: Sentiment analysis of restaurant reviews holds practical value for understanding customer opinions, improving restaurant services, and making informed business decisions. This script demonstrates the process of preprocessing text, training machine learning models, and evaluating their performance in sentiment analysis.

### Sentiment analysis plays a crucial role in extracting valuable insights from unstructured text data, contributing to enhanced customer experiences and data-driven decision-making in the restaurant industry.

Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from string import punctuation
from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

Load the dataset


In [ ]:
review=pd.read_csv("https://raw.githubusercontent.com/Timmapuram-Karthik/Edunet-IBM-Restaurant-Sentiment/main/Restaurant_Reviews.tsv",sep="\t")

Display the loaded dataset

In [ ]:
review

Check for any missing values in the dataset

In [ ]:
review.isnull().sum()

Display the first few rows of the dataset

In [ ]:
review.head()

Display the last few rows of the dataset

In [ ]:
review.tail()

Preprocess the data

In [ ]:
corpus=[]
stopwords=list(STOP_WORDS)
stopwords_to_remove=["n‘t","n't","n’t","not"]
stopwords=[word for word in stopwords if word not in stopwords_to_remove]

In [ ]:
for i in range(review.shape[0]):
    data=review.iloc[i,0]
    data=data.lower()
    data=data.split()
    ps=PorterStemmer()
    data=[ps.stem(word) for word in data if not word in set(stopwords)]
    data=' '.join(data)
    corpus.append(data)

Create Count vectorizer and transform the text data into a numerical format

In [ ]:
cv=CountVectorizer(max_features=1500, ngram_range=(1, 2), stop_words='english')
x=cv.fit_transform(corpus).toarray()
y=review['Liked'].values

Split the data into training and testing sets

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2)

Define models for sentiment analysis

In [ ]:
models={'MultinomialNB':MultinomialNB(),
        'RandomForestClassifier': RandomForestClassifier(n_estimators=100,max_depth=4),
        'GradientBoostingClassifier' : GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
        'xgb_classifier' : XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3,random_state=42),
        'SVC': SVC(kernel='linear')}

Initialize variables to track the best model and its accuracy

In [ ]:
best_model = None
best_accuracy = 0

Dictionary to store evaluation metrics for each model

In [ ]:
score={}

Loop through each model, train, and evaluate and check if current model has the best accuracy so far

In [ ]:
for model_name,model in models.items():
    model.fit(x_train,y_train)
    train_score=model.score(x_train,y_train)
    test_score=model.score(x_test,y_test)
    y_pred=model.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    if ((train_score-test_score)*100)<= 5 and accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model_name
    score[model_name]={'Train Score':train_score, 'Test Score':test_score,'Accuracy':accuracy,'y_pred':y_pred}

Print evaluation metrics for each model

In [ ]:
for model_name, scores in score.items():
    print(f"Model: {model_name}")
    print(f"Train Score: {scores['Train Score']:.4f}")
    print(f"Test Score: {scores['Test Score']:.4f}")
    print(f"Accuracy: {scores['Accuracy']:.2f}\n")

Print the best model and its accuracy

In [ ]:
print(f"The best model is {best_model} with an accuracy of {best_accuracy*100:.2f}%")

Calculate and display the confusion matrix for the best model

In [ ]:
conf_matrix = confusion_matrix(y_test, score[best_model]['y_pred'])
print("Confusion Matrix:")
print(conf_matrix)

Create and display the classification report for the best model

In [ ]:
class_rep = classification_report(y_test, score[best_model]['y_pred'])
print("Classification Report:")
print(class_rep)